In [1]:
import pandas as pd
import time
import numpy as np
import gc

In [2]:
test= pd.read_csv('train_day_8.csv',nrows=10000000, parse_dates=['click_time'])

In [3]:
test.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second
0,48240,23,1,13,153,2017-11-08,NaN,0,8,0,0,0
1,45702,13,1,16,477,2017-11-08,NaN,0,8,0,0,0
2,123214,12,1,13,259,2017-11-08,NaN,0,8,0,0,0
3,98052,15,1,19,265,2017-11-08,NaN,0,8,0,0,0
4,97903,6,1,18,125,2017-11-08,NaN,0,8,0,0,0


## Creat Features

###  1. Combination counting

In [4]:
print('grouping by ip-hour combination...')
gp = test[['ip','hour','channel']].groupby(by=['ip','hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_tcount'})
test= test.merge(gp, on=['ip','hour'], how='left')
del gp
gc.collect()

grouping by ip-hour combination...


51

In [5]:
test.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second,ip_tcount
0,48240,23,1,13,153,2017-11-08,NaN,0,8,0,0,0,1875
1,45702,13,1,16,477,2017-11-08,NaN,0,8,0,0,0,235
2,123214,12,1,13,259,2017-11-08,NaN,0,8,0,0,0,44
3,98052,15,1,19,265,2017-11-08,NaN,0,8,0,0,0,669
4,97903,6,1,18,125,2017-11-08,NaN,0,8,0,0,0,105


In [6]:
print('grouping by ip-app combination...')
gp = test[['ip', 'app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_count'})
test= test.merge(gp, on=['ip','app'], how='left')
del gp
gc.collect()

grouping by ip-app combination...


86

In [7]:
test.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second,ip_tcount,ip_app_count
0,48240,23,1,13,153,2017-11-08,NaN,0,8,0,0,0,1875,58
1,45702,13,1,16,477,2017-11-08,NaN,0,8,0,0,0,235,12
2,123214,12,1,13,259,2017-11-08,NaN,0,8,0,0,0,44,30
3,98052,15,1,19,265,2017-11-08,NaN,0,8,0,0,0,669,143
4,97903,6,1,18,125,2017-11-08,NaN,0,8,0,0,0,105,2


In [8]:
print('grouping by ip-app-os combination...')
gp = test[['ip','app', 'os', 'channel']].groupby(by=['ip', 'app', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_os_count'})
test = test.merge(gp, on=['ip','app', 'os'], how='left')
del gp
gc.collect()

grouping by ip-app-os combination...


102

### 2. Adding features with var and mean hour

In [9]:
print('grouping by : ip_chl_var_hour')
gp = test[['ip','hour','channel']].groupby(by=['ip','channel'])[['hour']].var().reset_index().rename(index=str, columns={'hour': 'ip_h_chan_var'})
test = test.merge(gp, on=['ip','channel'], how='left')
del gp
gc.collect()

grouping by : ip_chl_var_hour


92

In [10]:
print('grouping by : ip_app_os_var_hour')
gp = test[['ip','app', 'os', 'hour']].groupby(by=['ip', 'app', 'os'])[['hour']].var().reset_index().rename(index=str, columns={'hour': 'ip_app_os_var'})
test = test.merge(gp, on=['ip','app', 'os'], how='left')
del gp
gc.collect()

grouping by : ip_app_os_var_hour


111

In [11]:
print('grouping by : ip_app_chl_mean_hour')
gp = test[['ip','app', 'channel','hour']].groupby(by=['ip', 'app', 'channel'])[['hour']].mean().reset_index().rename(index=str, columns={'hour': 'ip_app_channel_mean_hour'})
test = test.merge(gp, on=['ip','app', 'channel'], how='left')
del gp
gc.collect()

grouping by : ip_app_chl_mean_hour


111

In [12]:
test.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second,ip_tcount,ip_app_count,ip_app_os_count,ip_h_chan_var,ip_app_os_var,ip_app_channel_mean_hour
0,48240,23,1,13,153,2017-11-08,NaN,0,8,0,0,0,1875,58,15,0.830099,0.828571,0.847826
1,45702,13,1,16,477,2017-11-08,NaN,0,8,0,0,0,235,12,2,0.696970,0.000000,0.750000
2,123214,12,1,13,259,2017-11-08,NaN,0,8,0,0,0,44,30,5,1.000000,0.700000,1.000000
3,98052,15,1,19,265,2017-11-08,NaN,0,8,0,0,0,669,143,25,0.545455,0.540000,0.444444
4,97903,6,1,18,125,2017-11-08,NaN,0,8,0,0,0,105,2,2,0.000000,0.000000,0.000000


In [13]:
print("vars and data type: ")
test.info()
test['ip_tcount'] = test['ip_tcount'].astype('uint16')
test['ip_app_count'] = test['ip_app_count'].astype('uint16')
test['ip_app_os_count'] = test['ip_app_os_count'].astype('uint16')


vars and data type: 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000000 entries, 0 to 9999999
Data columns (total 18 columns):
ip                          int64
app                         int64
device                      int64
os                          int64
channel                     int64
click_time                  datetime64[ns]
attributed_time             object
is_attributed               int64
day                         int64
hour                        int64
minute                      int64
second                      int64
ip_tcount                   int64
ip_app_count                int64
ip_app_os_count             int64
ip_h_chan_var               float64
ip_app_os_var               float64
ip_app_channel_mean_hour    float64
dtypes: datetime64[ns](1), float64(3), int64(13), object(1)
memory usage: 1.4+ GB


### 3. Confidence Rates for is_attributed

in the test set, we do not know the value of is_attributed, so we can just set the rate of is_attributed =0

In [14]:
test['ip_rate']= 0
test.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,minute,second,ip_tcount,ip_app_count,ip_app_os_count,ip_h_chan_var,ip_app_os_var,ip_app_channel_mean_hour,ip_rate
0,48240,23,1,13,153,2017-11-08,NaN,0,8,0,0,0,1875,58,15,0.830099,0.828571,0.847826,0
1,45702,13,1,16,477,2017-11-08,NaN,0,8,0,0,0,235,12,2,0.696970,0.000000,0.750000,0
2,123214,12,1,13,259,2017-11-08,NaN,0,8,0,0,0,44,30,5,1.000000,0.700000,1.000000,0
3,98052,15,1,19,265,2017-11-08,NaN,0,8,0,0,0,669,143,25,0.545455,0.540000,0.444444,0
4,97903,6,1,18,125,2017-11-08,NaN,0,8,0,0,0,105,2,2,0.000000,0.000000,0.000000,0


In [15]:
test['app_rate']= 0
test['device_rate']= 0
test['os_rate']= 0
test['channel_rate']= 0
test['app_channel_rate']= 0
test['app_os_rate']= 0
test['app_device_rate']= 0
test['os_channel_rate']= 0
test['device_channel_rate']= 0
test['device_os_rate']= 0

In [16]:
test.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,...,app_rate,device_rate,os_rate,channel_rate,app_channel_rate,app_os_rate,app_device_rate,os_channel_rate,device_channel_rate,device_os_rate
0,48240,23,1,13,153,2017-11-08,NaN,0,8,0,...,0,0,0,0,0,0,0,0,0,0
1,45702,13,1,16,477,2017-11-08,NaN,0,8,0,...,0,0,0,0,0,0,0,0,0,0
2,123214,12,1,13,259,2017-11-08,NaN,0,8,0,...,0,0,0,0,0,0,0,0,0,0
3,98052,15,1,19,265,2017-11-08,NaN,0,8,0,...,0,0,0,0,0,0,0,0,0,0
4,97903,6,1,18,125,2017-11-08,NaN,0,8,0,...,0,0,0,0,0,0,0,0,0,0


## 4. How long the time till the next click

In [17]:
GROUP_BY_NEXT_CLICKS = [
    
    # V1
    {'groupby': ['ip']},
    {'groupby': ['ip', 'app']},
    {'groupby': ['ip', 'channel']},
    {'groupby': ['ip', 'os']}
]

# Calculate the time to next click for each group
for spec in GROUP_BY_NEXT_CLICKS:
    
    # Name of new feature
    new_feature = '{}_nextClick'.format('_'.join(spec['groupby']))    
    
    # Unique list of features to select
    all_features = spec['groupby'] + ['click_time']
    
    # Run calculation
    print(f">> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}")
    test[new_feature] = test[all_features].groupby(spec['groupby']).click_time.transform(lambda x: x.diff().shift(-1)).dt.seconds



>> Grouping by ['ip'], and saving time to next click in: ip_nextClick
>> Grouping by ['ip', 'app'], and saving time to next click in: ip_app_nextClick
>> Grouping by ['ip', 'channel'], and saving time to next click in: ip_channel_nextClick
>> Grouping by ['ip', 'os'], and saving time to next click in: ip_os_nextClick


In [18]:
test.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,...,app_channel_rate,app_os_rate,app_device_rate,os_channel_rate,device_channel_rate,device_os_rate,ip_nextClick,ip_app_nextClick,ip_channel_nextClick,ip_os_nextClick
0,48240,23,1,13,153,2017-11-08,NaN,0,8,0,...,0,0,0,0,0,0,8.0,111.0,111.0,17.0
1,45702,13,1,16,477,2017-11-08,NaN,0,8,0,...,0,0,0,0,0,0,1.0,28.0,28.0,8.0
2,123214,12,1,13,259,2017-11-08,NaN,0,8,0,...,0,0,0,0,0,0,59.0,1525.0,4446.0,59.0
3,98052,15,1,19,265,2017-11-08,NaN,0,8,0,...,0,0,0,0,0,0,0.0,0.0,127.0,0.0
4,97903,6,1,18,125,2017-11-08,NaN,0,8,0,...,0,0,0,0,0,0,0.0,86.0,86.0,0.0


In [19]:
GROUP_BY_NEXT_CLICKS = [
    
    # V3
    {'groupby': ['ip', 'os', 'device']},
    {'groupby': ['ip', 'os', 'device', 'app']}
]

# Calculate the time to next click for each group
for spec in GROUP_BY_NEXT_CLICKS:
    
    # Name of new feature
    new_feature = '{}_nextClick'.format('_'.join(spec['groupby']))    
    
    # Unique list of features to select
    all_features = spec['groupby'] + ['click_time']
    
    # Run calculation
    print(f">> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}")
    test[new_feature] = test[all_features].groupby(spec['groupby']).click_time.transform(lambda x: x.diff().shift(-1)).dt.seconds

test.head()

>> Grouping by ['ip', 'os', 'device'], and saving time to next click in: ip_os_device_nextClick
>> Grouping by ['ip', 'os', 'device', 'app'], and saving time to next click in: ip_os_device_app_nextClick


,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,...,app_device_rate,os_channel_rate,device_channel_rate,device_os_rate,ip_nextClick,ip_app_nextClick,ip_channel_nextClick,ip_os_nextClick,ip_os_device_nextClick,ip_os_device_app_nextClick
0,48240,23,1,13,153,2017-11-08,NaN,0,8,0,...,0,0,0,0,8.0,111.0,111.0,17.0,17.0,111.0
1,45702,13,1,16,477,2017-11-08,NaN,0,8,0,...,0,0,0,0,1.0,28.0,28.0,8.0,8.0,28.0
2,123214,12,1,13,259,2017-11-08,NaN,0,8,0,...,0,0,0,0,59.0,1525.0,4446.0,59.0,59.0,4239.0
3,98052,15,1,19,265,2017-11-08,NaN,0,8,0,...,0,0,0,0,0.0,0.0,127.0,0.0,0.0,127.0
4,97903,6,1,18,125,2017-11-08,NaN,0,8,0,...,0,0,0,0,0.0,86.0,86.0,0.0,0.0,86.0


## 5. Clicks on app ad before & after

In [20]:
HISTORY_CLICKS = {
    'identical_clicks': ['ip', 'app', 'device', 'os', 'channel'],
    'app_clicks': ['ip', 'app']
}

# Go through different group-by combinations
for fname, fset in HISTORY_CLICKS.items():
    
    # Clicks in the past
    test['prev_'+fname] = test. \
        groupby(fset). \
        cumcount(). \
        rename('prev_'+fname)
        
    # Clicks in the future
    test['future_'+fname] = test.iloc[::-1]. \
        groupby(fset). \
        cumcount(). \
        rename('future_'+fname).iloc[::-1]

# Count cumulative subsequent clicks
test.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,...,ip_nextClick,ip_app_nextClick,ip_channel_nextClick,ip_os_nextClick,ip_os_device_nextClick,ip_os_device_app_nextClick,prev_identical_clicks,future_identical_clicks,prev_app_clicks,future_app_clicks
0,48240,23,1,13,153,2017-11-08,NaN,0,8,0,...,8.0,111.0,111.0,17.0,17.0,111.0,0,8,0,57
1,45702,13,1,16,477,2017-11-08,NaN,0,8,0,...,1.0,28.0,28.0,8.0,8.0,28.0,0,1,0,11
2,123214,12,1,13,259,2017-11-08,NaN,0,8,0,...,59.0,1525.0,4446.0,59.0,59.0,4239.0,0,2,0,29
3,98052,15,1,19,265,2017-11-08,NaN,0,8,0,...,0.0,0.0,127.0,0.0,0.0,127.0,0,4,0,142
4,97903,6,1,18,125,2017-11-08,NaN,0,8,0,...,0.0,86.0,86.0,0.0,0.0,86.0,0,1,0,1


In [21]:
test.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,hour,...,ip_nextClick,ip_app_nextClick,ip_channel_nextClick,ip_os_nextClick,ip_os_device_nextClick,ip_os_device_app_nextClick,prev_identical_clicks,future_identical_clicks,prev_app_clicks,future_app_clicks
0,48240,23,1,13,153,2017-11-08,NaN,0,8,0,...,8.0,111.0,111.0,17.0,17.0,111.0,0,8,0,57
1,45702,13,1,16,477,2017-11-08,NaN,0,8,0,...,1.0,28.0,28.0,8.0,8.0,28.0,0,1,0,11
2,123214,12,1,13,259,2017-11-08,NaN,0,8,0,...,59.0,1525.0,4446.0,59.0,59.0,4239.0,0,2,0,29
3,98052,15,1,19,265,2017-11-08,NaN,0,8,0,...,0.0,0.0,127.0,0.0,0.0,127.0,0,4,0,142
4,97903,6,1,18,125,2017-11-08,NaN,0,8,0,...,0.0,86.0,86.0,0.0,0.0,86.0,0,1,0,1


In [22]:
test.to_csv(path_or_buf='test_day_8_feature.csv', index = False)